<a href="https://colab.research.google.com/github/edcalderin/LLM_Tech/blob/master/Advanced_RAG_From_Theory_to_LlamaIndex_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Retrieval-Augmented Generation: From Theory to LlamaIndex Implementation

https://medium.com/data-science/advanced-retrieval-augmented-generation-from-theory-to-llamaindex-implementation-4de1464a9930

## Installing modules and setting up api keys

In [1]:
!pip install -qU llama-index weaviate-client llama-index-vector-stores-weaviate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.2/597.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 kB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curren

In [2]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## Implementing Naive RAG with LlamaIndex

In [3]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.settings import Settings

# 1. Define embeddings and model

Settings.llm = OpenAI(model="gpt-4o")
Settings.embed_model = OpenAIEmbedding()

In [4]:
!mkdir -p 'data'
!wget https://raco.cat/index.php/ECT/article/download/89018/133467/133672 -O 'data/Tsunami-de-2004.pdf'

--2025-08-01 20:15:01--  https://raco.cat/index.php/ECT/article/download/89018/133467/133672
Resolving raco.cat (raco.cat)... 84.88.27.11
Connecting to raco.cat (raco.cat)|84.88.27.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2247099 (2.1M) [application/pdf]
Saving to: ‘data/Tsunami-de-2004.pdf’

data/Tsunami-de-200 100%[===================>]   2.14M  1.64MB/s    in 1.3s    

2025-08-01 20:15:05 (1.64 MB/s) - ‘data/Tsunami-de-2004.pdf’ saved [2247099/2247099]



In [5]:
# 2. Load data

from llama_index.core import SimpleDirectoryReader

directory_reader = SimpleDirectoryReader(input_dir="data")
documents = directory_reader.load_data()

print(len(documents), "document(s) in this directory")

13 document(s) in this directory


In [6]:
# 3. Chunk documents into Nodes

from llama_index.core.node_parser import SimpleNodeParser

node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)
nodes: list = node_parser.get_nodes_from_documents(documents)

print(len(nodes))

print(nodes[:2])

16
[TextNode(id_='5cb4ffe9-2175-4906-99cc-0a8feeebf045', embedding=None, metadata={'page_label': '1', 'file_name': 'Tsunami-de-2004.pdf', 'file_path': '/content/data/Tsunami-de-2004.pdf', 'file_type': 'application/pdf', 'file_size': 2247099, 'creation_date': '2025-08-01', 'last_modified_date': '2025-08-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0f6c34a9-4db3-436e-a12a-3c334ca3c957', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'Tsunami-de-2004.pdf', 'file_path': '/content/data/Tsunami-de-2004.pdf', 'file_type': 'application/pdf', 'file_size': 2247099, 'creation_date': '2025-08-01', 'last_modified_date': '2025-08-01'}, hash='7beeec9affd690d2678e359951b84d3e0f0a03

In [13]:
# 4. Build index

from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
import weaviate

client = weaviate.WeaviateClient(
    embedded_options=weaviate.embedded.EmbeddedOptions()
)

client.connect()

INDEX_NAME: str = "Rag"

vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name=INDEX_NAME)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(nodes, storage_context=storage_context)

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 3721


In [12]:
# client.close()

In [14]:
# 5. Setup query engine

query_engine = index.as_query_engine()

In [18]:
# 6. Run a naive RAG

query_engine.query("Cuando ocurrio el tsunami?").response

'El tsunami ocurrió el 26 de diciembre de 2004.'

In [21]:
query_engine.query("Cual fue el pais mas afectado?").response

'El país más afectado no se menciona específicamente en el contexto proporcionado. Sin embargo, se indica que el terremoto y posterior tsunami afectaron a más de 10 países del sudeste asiático y de África oriental, con un balance de 283.100 víctimas mortales y más de 14.100 desaparecidos.'

In [22]:
query_engine.query("Cual fue el pais del sudeste asiatico mas afectado?").response

'El país del sudeste asiático más afectado fue Indonesia.'

In [23]:
query_engine.query("Cuantas personas murieron en Indonesia?").response

'Al menos 235.800 personas murieron en Indonesia.'

In [26]:
query_engine.query("De cuantos metros de altura fue la ola que ataco a Indonesia?").response

'La ola que atacó a Indonesia, específicamente en el área de Phuket, Tailandia, tuvo una altura de 3-5 metros.'

In [27]:
query_engine.query("Cuanto tardo el volver el agua al mar en Indonesia?").response

'El contexto no proporciona información específica sobre cuánto tiempo tardó el agua en volver al mar en Indonesia después del tsunami del 26 de diciembre de 2004.'

## Implementing Advanced RAG with LlamaIndex

In [28]:
from llama_index.core.node_parser import SentenceWindowNodeParser # Instead of SimpleNodeParser

node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text"
)

nodes: list = node_parser.get_nodes_from_documents(documents)

print(len(nodes))

print(nodes[:2])

279
[TextNode(id_='fccdd13c-418f-4ecb-98b9-431b0d9e05a4', embedding=None, metadata={'page_label': '1', 'file_name': 'Tsunami-de-2004.pdf', 'file_path': '/content/data/Tsunami-de-2004.pdf', 'file_type': 'application/pdf', 'file_size': 2247099, 'creation_date': '2025-08-01', 'last_modified_date': '2025-08-01', 'window': 'INTRODUCCIÓN\nEl 26 de diciembre de 2004 a las 00:58:53 T.U,\n(07:58:53 AM, hora local) una hora más en la Pe-\nnínsula Ibérica se produjo uno de los sismos más\nimportantes y jamás registrados, en la región de la\nisla de Sumatra, al suroeste de Banda Aceh, con\nuna magnitud estimada de 9.3 (USGS-NEIC) y con\nuna profundidad de 30 kilómetros.  Como resultado\ndel terremoto se produjo un devastador tsunami y el\nnivel de mar mar experimentó una elevación de va-\nrios metros en las costas de Indonesia y zonas cos-\nteras situadas a miles de kilómetros del epicentro.\n El sismo del 26 de diciembre de 2004 es el cuarto\nterremoto más grande del mundo desde 1900 y el más\ngr

In [29]:
from llama_index.core.postprocessor import (
    MetadataReplacementPostProcessor
)

client.connect()

INDEX_NAME: str = "Rag"

postprocessor = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)

vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name=INDEX_NAME)

storage_context = StorageContext.from_defaults(
    vector_store=vector_store
)

index = VectorStoreIndex(nodes, storage_context=storage_context)

query_engine = index.as_query_engine(
    node_postprocessors = [postprocessor]
)

In [30]:
query_engine.query("Cuando ocurrio el tsunami?").response

'El tsunami ocurrió el 26 de diciembre de 2004.'

In [31]:
query_engine.query("Cual fue el pais mas afectado?").response

'El país más afectado fue Indonesia, con al menos 235,800 personas afectadas entre muertos y desaparecidos.'

In [33]:
query_engine.query("De cuantos metros de altura fue la ola que ataco a Indonesia?").response

'La información proporcionada no menciona específicamente la altura de la ola que atacó a Indonesia.'

In [34]:
query_engine.query("Cuanto tardo el volver el agua al mar en Indonesia?").response

'La información proporcionada no especifica cuánto tiempo tardó el agua en volver al mar en Indonesia después del tsunami de 2004.'

## Retrieval optimization example: Hybrid search

In [35]:
query_engine = index.as_query_engine(
    node_postprocessors = [postprocessor],
    vector_store_query_mode = "hybrid",
    alpha=.5
)

In [36]:
query_engine.query("Cuando ocurrio el tsunami?").response

'El tsunami ocurrió el 26 de diciembre de 2004.'

In [37]:
query_engine.query("Cual fue el pais mas afectado?").response

'El país más afectado por el tsunami de 2004, en términos de número de muertos, fue Indonesia, específicamente en la región de Sumatra, donde el tsunami alcanzó alturas de hasta 30 metros.'

In [38]:
query_engine.query("De cuantos metros de altura fue la ola que ataco a Indonesia?").response

'La ola que atacó a Indonesia alcanzó alturas de 30 metros a lo largo de la costa occidental de Sumatra.'

In [39]:
query_engine.query("Cuanto tardo el volver el agua al mar en Indonesia?").response

'La información proporcionada no especifica cuánto tiempo tardó el agua en volver al mar en Indonesia después del tsunami de 2004.'

Nice job!!

## Post-retrieval optimization example: Re-ranking

In [58]:
from llama_index.core.postprocessor import SentenceTransformerRerank
from google.colab import userdata
import os

os.environ["HF_TOKEN"] = userdata.get('HUGGING_FACE_ACCESS_TOKEN')

# Defining reranker model
rerank = SentenceTransformerRerank(
    top_n=2,
    model="BAAI/bge-reranker-base"
)

# Add reramger tp query engine

query_engine = index.as_query_engine(
    similarity_top_k = 6,
    node_postprocessors = [rerank],
    vector_store_query_mode = "hybrid",
    alpha=.5
)

In [59]:
query_engine.query("Cuando ocurrio el tsunami?").response

'El tsunami ocurrió el 26 de diciembre de 2004.'

In [60]:
query_engine.query("Cual fue el pais mas afectado?").response

'El país más afectado por el tsunami de 2004, según el número de víctimas mortales y desplazados mencionados, fue Indonesia. Sin embargo, en el contexto proporcionado, se mencionan otros países afectados, pero no se especifica cuál fue el más afectado en términos absolutos.'

In [61]:
query_engine.query("De cuantos metros de altura fue la ola que ataco a Indonesia?").response

'La información proporcionada no menciona específicamente la altura de la ola que atacó a Indonesia.'

In [62]:
query_engine.query("Cuanto tardo el volver el agua al mar en Indonesia?").response

'La información proporcionada no especifica cuánto tiempo tardó el agua en volver al mar en Indonesia después del tsunami de 2004.'